<a href="https://colab.research.google.com/github/fadel11-hub/Tim-Coba-Coba/blob/main/Final_Model_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Whisper

In [6]:
!pip install openai-whisper

# Klasifikasi

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# --- 1. Load dataset ---
df = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")  # ganti dengan file datasetmu
texts = df["text"].astype(str).values
labels = df["label"].values

# --- 2. Encode label ke angka ---
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# --- 3. Tokenisasi dan padding ---
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_length = 20  # panjang maksimum kalimat
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

vocab_size = len(tokenizer.word_index) + 1  # +1 untuk padding
embedding_dim = 50

# --- 4. Split data ---
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, encoded_labels, test_size=0.2, random_state=42
)

# --- 5. Bangun model LSTM ---
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# --- 6. Latih model ---
history = model.fit(
    X_train, y_train, epochs=10, batch_size=16,
    validation_data=(X_test, y_test)
)

# --- 7. Simpan model dan tokenizer ---
model.save("lstm_model.h5")
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - accuracy: 0.4939 - loss: 0.6947 - val_accuracy: 0.5366 - val_loss: 0.6926
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.6244 - loss: 0.6912 - val_accuracy: 0.4634 - val_loss: 0.6931
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.5562 - loss: 0.6917 - val_accuracy: 0.4634 - val_loss: 0.6963
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.5065 - loss: 0.6930 - val_accuracy: 0.4634 - val_loss: 0.6951
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.5133 - loss: 0.6908 - val_accuracy: 0.4634 - val_loss: 0.6932
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.5126 - loss: 0.6868 - val_accuracy: 0.4634 - val_loss: 0.6756
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.6795 - loss: 0.6269 - val_accuracy: 0.9024 - val_loss: 0.3399
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.9935 - loss: 0.2012 - val_accuracy: 0.9512 - 

In [8]:
import whisper

model_whisper = whisper.load_model("base")


result = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/common_voice_id_41969591.mp3")
result3 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Tolong aku.m4a")
result4 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Saya menonton video.m4a")
result5 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat")
result6 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/aman")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [14]:
from tensorflow.keras.models import load_model
import pickle

# Load model & tokenizer
model = load_model("lstm_model.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Kalimat baru
seq = tokenizer.texts_to_sequences(result6['text'])
pad = pad_sequences(seq, maxlen=20, padding="post")

# Prediksi
pred_prob = model.predict(pad)[0][0]
pred_label = 1 if pred_prob >= 0.5 else 0
final_label = label_encoder.inverse_transform([pred_label])[0]

print("Kalimat:", result6['text'])
print("Probabilitas darurat:", pred_prob)
print("Prediksi Label:", final_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
Kalimat:  Saya belajar kodin.
Probabilitas darurat: 0.9983857
Prediksi Label: darurat


# Dependency

In [10]:
print('whisper', whisper.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('tensorflow', tf.__version__)
print('scikit-learn', sklearn.__version__)

whisper 20250625
pandas 2.2.2
numpy 2.0.2
tensorflow 2.18.0
scikit-learn 1.6.1


In [11]:
!pip freeze > requirements.txt

In [38]:

%%writefile requirements.txt
whisper=20250625
pandas=2.2.2
numpy=2.0.2
tensorflow=2.18.0
scikit-learn=1.6.1


Writing requirements.txt


In [40]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>